# PYNQ with Ardumoto motor driver shield 

Actuators such as D.C. Motors are one of the most useful peripherals in most of the projects. 
PYNQ can be interfaced with Sparkfun Ardumoto Shield to control two motors simultanoeouly. 

## Contents

* [Background](#Background)
* [Ardumoto Shield](#Ardumoto-Shield)
* [Connections](#Connections)
* [Setup](#Setup)
* [Motor Control](#Motor-Control)
* [Robotics Concepts](#Robotics-Concepts)


### Background

The simplest of all motors, DC motors turn when a DC voltage is applied across it. This kind of motor can be found in drones, power tools, and robots. A DC motor can change speend and direction depending on how much power is fed to it and in which direction.

The DC motor uses a magnetic field generated by the by an electromagnet to turn the armature of a motor. The electromagnet is activated by applying voltage, so when the power is on, the magnetic field it generates will cause the armature (a coil of wire) to generate its own nagmetic field, these fields push eachother away and cause the armature to spin.

To get the motor to spin the other way, we need to reverse the applied voltage, meaning the flow of current through the motor will be reversed. Unfortunately switching the direction of current from a controller is difficult. The processors use low current and voltages, plus they are usually disconnected from the motor to prevent inductive feedback distrupting their operation.

If only there was some kind of device that can help us control the power we supply to our DC motors…

##### How H-Bridges Work
An H-Bridge is a circuit that allows voltages to be applied across a load in either direction. Electric current flows from the source to ground, and many components need to be oriented according to the direction of current to work as expected. An H-Bridge is a circuit built to change the direction of the voltage and thus the current flowing to a load.

An H-Bridge is made up of four switches: two in series, and two in parallel, with the load placed in between the switches. In this configuration the circuit takes an “H” shape.

![alt text](data/H-Bridge.gif)

In order to change the direction of the voltage supplied, the H-Bridge controls the switches that deliver power to the load (M). 

Looking at the diagram, if we close S1 and S4 while leaving the rest open, the voltage will be applied from left to right across the motor. 

If S2 and S3 are closed instead and the others open, the voltage will be applied from right to left.

### Ardumoto Shield

[Ardumoto](https://www.sparkfun.com/products/14180).
Ardumoto supports upto two DC motor.
It uses H-Bridge to control the speed and direction of each motor.

Ardumoto shield sits on top of the arduino headers on the PYNQ board and uses an external battery to power the motors.

The Ardumoto shield can be mounted on the PYNQ Board similar to PYNQ arduino shield:

Motor A and Motor B are connected as below to the arduino pins:

| Pin number | Functionality                    |
|------------|----------------------------------|
| 2          | Direction control for motor A    |
| 3          | PWM control (speed) for motor A  |
| 4          | Direction control for motor B    |
| 11         | PWM control (speed) for motor B  |

### Connections

#### Connecting Ardumot to PYNQ Board

![alt text](data/PYNQ_Ardumoto.jpg)

#### Connecting the Motors to Ardumoto Shield  

![alt text](data/ardumoto_wiring_cdc.jpg)

#### Connecting the Battery to Ardumoto Shield 

![alt text](data/Ardumoto_Battery.jpg)

### Setup

This training demonstrates how C code can be invoked from within the Python notebook to control a peripheral.
But before we start exploring that, as always we need to make sure the base overlay is loaded to the board.

We will also define some global constants that will be useful to control the motor.


In [20]:
# Setup base PYNQ environment
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

# Import sleep
from time import sleep

# Define constants to control motor direction
MOTOR_A = 0
MOTOR_B = 1
POLAR_DEFAULT = 0
POLAR_REVERSE = 1
CLOCKWISE = 0
ANTICLOCKWISE = 1

In [21]:
%%microblaze base.ARDUINO

#include "xio_switch.h"
#include "gpio.h"
#include "timer.h"

#define DEFAULT_PERIOD 625998
#define DEFAULT_DUTY 312998

#define DIR_A_PIN 2
#define PWM_A_PIN 3
#define DIR_B_PIN 4
#define PWM_B_PIN 11

typedef enum motor {
    MOTOR_A = 0,
    MOTOR_B = 1,
}motor_e;

static unsigned int pol_a = 0, pol_b = 0;
static unsigned int dir_a = 0, dir_b = 0;
static unsigned int duty_a = 50, duty_b = 50;

static timer timer_a;
static timer timer_b;
static gpio gpio_a;
static gpio gpio_b;

unsigned int init_ardumoto(){    
    timer_a = timer_open_device(0);
    timer_b = timer_open_device(5);
    set_pin(PWM_A_PIN, PWM0);
    set_pin(PWM_B_PIN, PWM5);
    gpio_a = gpio_open(DIR_A_PIN);
    gpio_b = gpio_open(DIR_B_PIN);
    gpio_set_direction(gpio_a, GPIO_OUT);
    gpio_set_direction(gpio_b, GPIO_OUT);
    return 0;
}

void configure_polar(unsigned int motor, unsigned int polarity){
    if (motor == MOTOR_A) {
        pol_a = polarity;
    }else if (motor == MOTOR_B) {
        pol_b = polarity;
    }
}

void set_direction(unsigned int motor, unsigned int direction){
    if (motor == MOTOR_A){
        dir_a = (direction)? pol_a : !pol_a;
    }
    else if (motor == MOTOR_B){
        dir_b = (direction)? pol_b : !pol_b;
    }
}

void set_speed(unsigned int motor, unsigned int speed){
    if (motor == MOTOR_A) {
        duty_a = speed;
    } else if (motor == MOTOR_B) {
        duty_b = speed;
    }
}

void run(unsigned int motor){
    if (motor == MOTOR_A) {
        gpio_write(gpio_a, dir_a);
        timer_pwm_generate(timer_a, DEFAULT_PERIOD, 
                           duty_a*DEFAULT_PERIOD/100);
    }else if(motor == MOTOR_B) {
        gpio_write(gpio_b, dir_b);
        timer_pwm_generate(timer_b, DEFAULT_PERIOD, 
                           duty_b*DEFAULT_PERIOD/100);
    }
}

void stop(unsigned int motor){
    if (motor == MOTOR_A) {
        timer_pwm_stop(timer_a);
    }else if (motor == MOTOR_B){
        timer_pwm_stop(timer_b);
    }
}

In [22]:
init_ardumoto()

configure_polar(MOTOR_A, POLAR_DEFAULT) 
configure_polar(MOTOR_B, POLAR_DEFAULT)

### Motor Control

### Run Both Motors Clockwise at Low Speed

Lets use Ardumoto shield to run both motors clockwise at low speed (10%)


In [23]:
from time import sleep

set_direction(MOTOR_A, CLOCKWISE)
set_direction(MOTOR_B, CLOCKWISE)
set_speed(MOTOR_A, 8)
set_speed(MOTOR_B, 8)

print('Running Motors A and B at 10% speed for 3 seconds')
run(MOTOR_A)
run(MOTOR_B)

sleep(3)

print('Stopping Motors A and B')
stop(MOTOR_A)
stop(MOTOR_B)
sleep(3)

Running Motors A and B at 10% speed for 3 seconds
Stopping Motors A and B


### Run Both Motors Counterclockwise at Low Speed

Lets use Ardumoto shield to run both motors counterclockwise at low speed (10%)

In [24]:
from time import sleep

set_direction(MOTOR_A, ANTICLOCKWISE)
set_direction(MOTOR_B, ANTICLOCKWISE)
set_speed(MOTOR_A, 10)
set_speed(MOTOR_B, 10)

print('Running Motors A and B at 10% speed for 3 seconds')
run(MOTOR_A)
run(MOTOR_B)

sleep(3)

print('Stopping Motors A and B')
stop(MOTOR_A)
stop(MOTOR_B)
sleep(3)

Running Motors A and B at 10% speed for 3 seconds
Stopping Motors A and B


### Run Both Motors Clockwise at High Speed

Lets use Ardumoto shield to run both motors clockwise at high speed (90%)

WARNING: Motors drive lot of current at high speed so always use with external Battery. 


In [25]:
from time import sleep

set_direction(MOTOR_A, CLOCKWISE)
set_direction(MOTOR_B, CLOCKWISE)
set_speed(MOTOR_A, 90)
set_speed(MOTOR_B, 90)

print('Running Motors A and B at 90% speed for 3 seconds')
run(MOTOR_A)
run(MOTOR_B)

sleep(3)

print('Stopping Motors A and B')
stop(MOTOR_A)
stop(MOTOR_B)
sleep(3)

Running Motors A and B at 90% speed for 3 seconds
Stopping Motors A and B


### Run Both Motors in opposite direction

Lets use Ardumoto shield to run motor in opposite direction at low speed

In [26]:
from time import sleep

set_direction(MOTOR_A, ANTICLOCKWISE)
set_direction(MOTOR_B, CLOCKWISE)
set_speed(MOTOR_A, 10)
set_speed(MOTOR_B, 10)

print('Running Motors A and B at 90% speed for 3 seconds')
run(MOTOR_A)
run(MOTOR_B)

sleep(3)

print('Stopping Motors A and B')
stop(MOTOR_A)
stop(MOTOR_B)
sleep(3)

Running Motors A and B at 90% speed for 3 seconds
Stopping Motors A and B


### Robotics Concepts 

A D.C. Motor assembly can be used with a chassis to drive and steer a robot in different directions.
Depending on the relative speed and direction of the two motors, we can achive different angles of turn.

Let us first define some helper functions for this:

* Move Forwards
* Move Backwards
* Turn left
* Turn Right


In [27]:
def robot_GoForward (speed,time):
    set_direction(MOTOR_A, CLOCKWISE)
    set_direction(MOTOR_B, CLOCKWISE)
    set_speed(MOTOR_A, speed)
    set_speed(MOTOR_B, speed)
    run(MOTOR_A)
    run(MOTOR_B)
    sleep(time)
    stop(MOTOR_A)
    stop(MOTOR_B)
    sleep(1)
    
def robot_GoBackward (speed,time):
    set_direction(MOTOR_A, ANTICLOCKWISE)
    set_direction(MOTOR_B, ANTICLOCKWISE)
    set_speed(MOTOR_A, speed)
    set_speed(MOTOR_B, speed)
    run(MOTOR_A)
    run(MOTOR_B)
    sleep(time)
    stop(MOTOR_A)
    stop(MOTOR_B)
    sleep(1)
    
def robot_TurnLeft (speed,time):
    set_direction(MOTOR_A, CLOCKWISE)
    set_direction(MOTOR_B, ANTICLOCKWISE)
    set_speed(MOTOR_A, speed)
    set_speed(MOTOR_B, speed)
    run(MOTOR_A)
    run(MOTOR_B)
    sleep(time)
    stop(MOTOR_A)
    stop(MOTOR_B)
    sleep(1)
    
def robot_TurnRight (speed,time):
    set_direction(MOTOR_A, ANTICLOCKWISE)
    set_direction(MOTOR_B, CLOCKWISE)
    set_speed(MOTOR_A, speed)
    set_speed(MOTOR_B, speed)
    run(MOTOR_A)
    run(MOTOR_B)
    sleep(time)
    stop(MOTOR_A)
    stop(MOTOR_B)
    sleep(1)